<a href="https://colab.research.google.com/github/amitaipat-create/Session2/blob/main/Another_copy_of_Mission_3_Practice_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practice 2: LangChain Agents with Tools

In [ ]:
%pip install langchain==0.3.27 langchain-openai==0.3.29 langchain-community==0.3.27 langchain-core pydantic==2.11.10 python-dotenv --quiet

In [ ]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from pydantic import BaseModel, Field
from typing import List

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

In [ ]:
# ============================================================================
# CONCEPT 1: create_tool_calling_agent() Setup
# Learning: Create an agent that can call tools automatically
# ============================================================================
print("\nCONCEPT 1: create_tool_calling_agent() Setup")
print("-" * 60)

# Create a simple tool
@tool
def get_skill_level(skill: str) -> str:
    """Get experience level for a skill"""
    levels = {
        "Python": "5 years",
        "FastAPI": "3 years",
        "PostgreSQL": "2 years"
    }
    return levels.get(skill, "No experience")

# Create prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use tools to answer questions."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Create agent
tools = [get_skill_level]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

result = agent_executor.invoke({"input": "What's the experience level for Python?"})
print(f"Agent response: {result['output']}")


CONCEPT 1: create_tool_calling_agent() Setup
------------------------------------------------------------
Agent response: The experience level for Python is 5 years.


In [ ]:
# ============================================================================
# EXERCISE 1: Create a Tool for Skill Matching
# Task: Create a tool that checks if a skill exists in a list
# Instructions:
#   1. Create a tool called 'check_skill' that takes two parameters:
#      - skill: str (the skill to check)
#      - skill_list: str (comma-separated skills like "Python, FastAPI, Docker")
#   2. Return "Found" if skill is in the list, "Not found" otherwise
#   3. Add it to the tools and test with the agent




In [ ]:
# ============================================================================
# CONCEPT 2: Custom Tool Creation with @tool Decorator
# Learning: Create custom tools for specific tasks
# ============================================================================
print("\nCONCEPT 2: Custom Tool Creation")
print("-" * 60)

@tool
def extract_skills(text: str) -> str:
    """Extract technical skills from text"""
    skills = []
    skill_keywords = ["Python", "FastAPI", "PostgreSQL", "Docker", "AWS"]
    for keyword in skill_keywords:
        if keyword.lower() in text.lower():
            skills.append(keyword)
    return ", ".join(skills) if skills else "No skills found"

@tool
def calculate_match_score(required_skills: str, candidate_skills: str) -> str:
    """Calculate match score between required and candidate skills"""
    required = set(required_skills.split(", "))
    candidate = set(candidate_skills.split(", "))
    matched = required.intersection(candidate)
    score = (len(matched) / len(required)) * 100 if required else 0
    return f"Match score: {score:.1f}% ({len(matched)}/{len(required)} skills matched)"

# Test tools
tools_list = [extract_skills, calculate_match_score]
test_agent = create_tool_calling_agent(llm, tools_list, prompt)
test_executor = AgentExecutor(agent=test_agent, tools=tools_list, verbose=False)

result = test_executor.invoke({
    "input": "Extract skills from 'Python developer with FastAPI and Docker experience'"
})
print(f"Tool result: {result['output']}")


CONCEPT 2: Custom Tool Creation
------------------------------------------------------------
Tool result: The extracted skills are:
- Python
- FastAPI
- Docker


In [ ]:
# ============================================================================
# EXERCISE 2: Try It Out - Create Tools for Job Matching (DO IT LATER)
# Task: Create tools that work together for job matching
# Instructions:
#   1. Create a tool 'get_years_experience' that extracts years from text
#      Example: "5 years Python" -> returns "5"
#   2. Create a tool 'check_experience_requirement' that compares years
#      Takes: candidate_years (int), required_years (int)
#      Returns: "Meets requirement" or "Below requirement"
#   3. Create an agent with both tools and test with:
#      "Candidate has 3 years Python experience. Job requires 5 years. Check if they meet requirement"
# ============================================================================




In [ ]:
# ============================================================================
# CONCEPT 3: Agent Memory (ConversationBufferMemory)
# Learning: Give agents memory to remember previous conversations
# ============================================================================
print("\nCONCEPT 3: Agent Memory")
print("-" * 60)

# Create memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Update prompt to include memory
prompt_with_memory = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Remember previous conversation."),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Create agent with memory
agent_with_memory = create_tool_calling_agent(llm, tools_list, prompt_with_memory)
memory_executor = AgentExecutor(
    agent=agent_with_memory,
    tools=tools_list,
    memory=memory,
    verbose=False
)

# First question
result1 = memory_executor.invoke({"input": "Extract skills from 'Python developer'"})
print(f"Q1: {result1['output']}")

# Follow-up (agent remembers context)
result2 = memory_executor.invoke({"input": "What was the previous skill?"})
print(f"Q2: {result2['output']}")


CONCEPT 3: Agent Memory
------------------------------------------------------------


/tmp/ipython-input-353751966.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


Q1: The extracted skill from "Python developer" is: **Python**.
Q2: The previous skill extracted was **Python**.


In [ ]:
# ============================================================================
# EXERCISE 3: Try It Out - Memory in Action (EASY)
# Task: Create a simple 2-turn conversation where agent remembers
# Instructions:
#   1. Copy the memory setup from Concept 3 above (lines 136-156)
#   2. Create 2 simple exchanges:
#      - Turn 1: Ask "What is your name?"
#      - Turn 2: Ask "What did I say my name was?" (agent should remember!)
#   3. That's it! Just test that memory works
# ============================================================================




In [ ]:
# ============================================================================
# CONCEPT 4: Structured Outputs (Pydantic Models)
# Learning: Get structured data from agents using Pydantic
# ============================================================================
print("\nCONCEPT 4: Structured Outputs")
print("-" * 60)

class SkillAnalysis(BaseModel):
    skills: List[str] = Field(description="List of technical skills found")
    count: int = Field(description="Number of skills")
    primary_skill: str = Field(description="Most important skill")

# Use structured output
structured_llm = llm.with_structured_output(SkillAnalysis)

result = structured_llm.invoke(
    "Analyze this: 'Senior Python developer with FastAPI and PostgreSQL experience'"
)
print(f"Structured output: {result}")
print(f"Skills: {result.skills}")
print(f"Primary: {result.primary_skill}")


CONCEPT 4: Structured Outputs
------------------------------------------------------------
Structured output: skills=['Python', 'FastAPI', 'PostgreSQL'] count=3 primary_skill='Python'
Skills: ['Python', 'FastAPI', 'PostgreSQL']
Primary: Python


In [ ]:
# ============================================================================
# EXERCISE 4: Try It Out - Create MatchResult Model (EASY)
# Task: Create a Pydantic model for match results and use structured output
# Instructions:
#   1. Create a MatchResult class with:
#      - match_score: float (0-100)
#      - matched_skills: List[str]
#      - missing_skills: List[str]
#   2. Use it with structured_llm to analyze a job match
#   3. Test with: "Job requires: Python, FastAPI, PostgreSQL. Candidate has: Python, FastAPI."
# ============================================================================




In [ ]:
# ============================================================================
# CONCEPT 5: Tool Integration and Execution
# Learning: Agents automatically decide when and how to use tools
# ============================================================================
print("\nCONCEPT 5: Tool Integration")
print("-" * 60)

@tool
def get_job_requirements(job_title: str) -> str:
    """Get requirements for a job title"""
    requirements = {
        "Software Engineer": "Python, FastAPI, PostgreSQL, Docker",
        "Data Scientist": "Python, Machine Learning, SQL, Statistics"
    }
    return requirements.get(job_title, "Requirements not found")

all_tools = [get_job_requirements, extract_skills, calculate_match_score]
multi_tool_agent = create_tool_calling_agent(llm, all_tools, prompt)
multi_executor = AgentExecutor(agent=multi_tool_agent, tools=all_tools, verbose=True)

result = multi_executor.invoke({
    "input": "Get requirements for Software Engineer, then extract skills from 'Python developer with FastAPI', then calculate match score"
})
print(f"Multi-tool result: {result['output']}")


CONCEPT 5: Tool Integration
------------------------------------------------------------


> Entering new AgentExecutor chain...

Invoking: `get_job_requirements` with `{'job_title': 'Software Engineer'}`


Python, FastAPI, PostgreSQL, Docker
Invoking: `extract_skills` with `{'text': 'Python developer with FastAPI'}`


Python, FastAPI
Invoking: `calculate_match_score` with `{'required_skills': 'Python, FastAPI, PostgreSQL, Docker', 'candidate_skills': 'Python, FastAPI'}`


Match score: 50.0% (2/4 skills matched)The requirements for a Software Engineer include the following skills: **Python, FastAPI, PostgreSQL, Docker**.

The skills extracted from the text "Python developer with FastAPI" are: **Python, FastAPI**.

The match score between the required skills and the candidate skills is **50.0%** (2 out of 4 skills matched).

> Finished chain.
Multi-tool result: The requirements for a Software Engineer include the following skills: **Python, FastAPI, PostgreSQL, Docker**.

The skills ex

In [ ]:
# ============================================================================
# EXERCISE 5: Try It Out - Chain Multiple Tools (DO IT LATER)
# Task: Create an agent that uses multiple tools in sequence
# Instructions:
#   1. Create a tool 'get_skill_description' that returns description for a skill
#      Example: "Python" -> "High-level programming language for general-purpose programming"
#   2. Combine it with existing tools (extract_skills, calculate_match_score)
#   3. Ask agent to: extract skills, get descriptions, then calculate match
#   4. Agent should use tools in logical sequence automatically
# ============================================================================




In [ ]:
# ============================================================================
# CONCEPT 6: Agent Conversation Flow
# Learning: Maintain context across multiple exchanges (essential for interview agent)
# ============================================================================
print("\nCONCEPT 6: Agent Conversation Flow")
print("-" * 60)

# Reset memory for clean example
interview_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

interview_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an interview agent. Ask questions and remember responses."),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

interview_agent = create_tool_calling_agent(llm, [], interview_prompt)  # No tools for simple interview
interview_executor = AgentExecutor(
    agent=interview_agent,
    tools=[],
    memory=interview_memory,
    verbose=False
)

# Multi-turn conversation
print("Interview conversation:")
response1 = interview_executor.invoke({"input": "Ask me about my Python experience"})
print(f"Agent: {response1['output']}")

response2 = interview_executor.invoke({"input": "I have 5 years of Python experience"})
print(f"Agent: {response2['output']}")

response3 = interview_executor.invoke({"input": "What did I say about my experience?"})
print(f"Agent: {response3['output']}")  # Agent remembers!


CONCEPT 6: Agent Conversation Flow
------------------------------------------------------------
Interview conversation:
Agent: Sure! How long have you been working with Python, and what types of projects or applications have you developed using it?
Agent: That's great! In those 5 years, what types of projects or applications have you worked on? Are there any specific areas of Python that you specialize in, such as data analysis, web development, automation, or something else?
Agent: You mentioned that you have 5 years of Python experience.


In [ ]:
# ============================================================================
# EXERCISE 6: Try It Out - Memory + Tools Together (CAN DO IT LATER)
# Task: Create an agent with memory that uses tools across multiple turns
# Instructions:
#   1. Create agent with memory + tools (extract_skills, calculate_match_score)
#   2. First turn: Extract skills from candidate description
#   3. Second turn: Calculate match score (agent should remember extracted skills)
#   4. Third turn: Ask agent to summarize the match (should remember everything)
#

